---
# Машинное обучение для текстов
---

# Содержание <a id='Content'></a>

[Описание проекта](#Task_description)         
[Описание данных проекта](#Data_description)     

1. [Подготовка данных](#1)    
    1.1 [Открытие и изучение данных](#1.1)    
    1.2 [Очистка и лемматизация корпуса текстов ](#1.2)     
    1.3 [Создание матрицы TF-IDF ](#1.3)    
    1.4 [Создание Embeddings на базе BERT](#1.4)
    
2. [Обучение](#2)    
    2.1 [Логистическая регрессия на TF-IDF](#2.1)    
    2.2 [LightGBM на признаках TF-IDF](#2.2)    
    2.3 [Логистическая регрессия на Embeddings из BERT](#2.3)    
    2.4 [LightGBM на эмбеддингах из BERT](#2.4)

3. [Выводы](#3)    

[Чек-лист проверки](#Check_list)    
[Записи для себя](#For_myself)

# [Описание проекта](#Content) <a id='Task_description'></a>

Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. Клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Необходимо подготовить модель, которая отклассифицирует комментарии на позитивные и негативные. В наличии размеченный набор данных с классом "токсичности" правок.

Мы стремимся построить модель со значением метрики качества *F1* не меньше 0.75. 

## [Описание данных](#Content) <a id='Data_description'></a>

Данные записаны в файле `toxic_comments.csv`. 

Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.


# 1. [Подготовка](#Content) <a id='1'></a>

Итак, тема интересная и сложная с позиции объема машинных операций. В ходе проекта мы постараемся выбирать типы данных, которые не будут расходовать много памяти и максимально упрощать машинные операции. 
Почитав слак стало понятно, что выполнять работу лучше не на своих мощностях.

Примерный порядок работы соответствует оглавлению.
Открываем датасет.

## 1.1 [Открытие и изучение данных](#Content) <a id='1.1'></a>

In [3]:
import pandas as pd
alt_path = 'https://code.s3.yandex.net/datasets/toxic_comments.csv'
path = '/datasets/toxic_comments.csv'
local_path = 'toxic_comments.csv'
toxic_comments = pd.read_csv(local_path)
toxic_comments.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [4]:
toxic_comments.sample(10).text

60467     "\nThe two of you both seem actively involved ...
110669    Re:disruption \n\nlook i'm sorry it's nothing ...
100037    As I said, an article can, and should, be edit...
156749    The submission was deleted because it had no c...
53978     Support – As you mention, the article title sh...
132267    yo, , if u r going 2 impersonate me, at least ...
71411     "\n\nSitush , really sweet  to see  you are ta...
14705     no abusing bots and stop being an idiot\nthe p...
115105                      the jews. Like, times a million
125305    Are you a moderator? \n\nIf you aren't a moder...
Name: text, dtype: object

In [5]:
toxic_comments.text[0]

"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"

In [6]:
toxic_comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [7]:
toxic_comments.toxic.value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

Видим две колонки в датасете как и сказано в описании:
- Текст
- его "токсичность" - целевой признак. Классы не сбалансированы    
В тексте присутствуют спецсимволы, отвечающие за перенос и сдвиг каретки в начало строки. Местами присутствуют "смайлы", восклицания и прочие сочетания символов-выражения эмоций. 
Нам придется провести предобработку текста.



In [17]:
%%time
corpus1 = toxic_comments['text'].values.astype('U') # формируем корпус
corpus = list(corpus1)

Wall time: 3.74 s


Коротко, что мы видим по ошибкам или мусору в корпусе текстов и что мы точно должны устранить:

| Ошибка       | Вид в тексте       | Исправление      |
| -------------|:------------------:| ----------------:|
| Знаки переноса строки    | \n    | Заменяем на " " |
| Апостроф с слешем     | Armenia\\'s |  Убираем слеш  |
| Знаки цитат  |   ""       |  Убираем   |
| URL линки    |   http://...       |  Убираем. полезной информации не несут   |
| IP адреса    |   70.100.229.154       |  Убираем  |
| Время и даты    |   04:28:57, August 19, 2007 (UTC)       |  Убираем  |
| Наборы символов    |   ==,!.        |  Убираем  |


## 1.2 [Очистка и лемматизация корпуса текстов](#Content) <a id='1.2'></a>

Немного экспериментов "для себя" в целях проверить подход.

In [8]:
import re

In [9]:
string = toxic_comments.text[0]
string

"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"

In [10]:
rex1 = r'(\d{2})[.-:](\d{2}),\w+(\d{2}),(\d{4})(\(UTC\))'
rex2 = r'(\d+)[/.-](\d{2})[/.-](\d{4})$'
rex3 = r'[(\d+)(\:)]|(\btalk\b|\bUTC\b)|((\d{1,3})\.(\d{1,3})\.(\d{1,3})\.(\d{1,3}))|(\n)'
rex4 = r'(\d{1,3})\.(\d{1,3})\.(\d{1,3})\.(\d{1,3})'
rex5 = r'(\btalk\b|\bUTC\b)'
re.sub(rex3,' ',string)

"Explanation Why the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the   page since I'm retired now.  .   .  .  "

Окончание экспериментов))

Пишем готовую функцию, которая:
1. Очищает текст от "лишней" информации, не имеющей отношения к "позитивности":
    - числа, даты, время, IP
    - Ссылки. Все равно контекст находится в рамках предложения
    - знаки и спецсимволы
2. Лемматизации текста

Декомпозицией слов, склеенных через апостроф пренебрежем    

In [11]:
import nltk
from nltk.stem import WordNetLemmatizer
# from pymystem3 import Mystem #После тестов отказался от конструкции

In [12]:
%%time
nltk.download('wordnet')

Wall time: 378 ms


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\pnedviga\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [13]:
%%time
nltk.download('averaged_perceptron_tagger')

Wall time: 17 ms


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\pnedviga\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [14]:
%%time
nltk.download('punkt')

Wall time: 46.9 ms


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pnedviga\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [15]:
def clear_text(text):    
    regex =r'[^a-zA-Z]|(\bUTC\b|(http://\S+))' # r'[^a-zA-Zа-яА-ЯёЁ]'
    cleaned_text = re.sub(regex, ' ', text.lower())    
    cleaned_text = cleaned_text.split()
    cleaned_text = " ".join(cleaned_text)
    return cleaned_text


'''
Создаем объект класса WordNetLemmatizer. Я его вынес за функцию, в которой он используется. 
Иначе функция создает его отдельно каждый раз, сильно замедляя работу алгоритма
'''
m = WordNetLemmatizer() 
def lemmatize(text):    
    lemm_list = m.lemmatize(text)
    lemm_text = "".join(lemm_list)        
    return lemm_text

In [18]:
test_text = corpus[15]
print(test_text)
clear_text(test_text) # Проверяем работоспособность

"

Juelz Santanas Age

In 2002, Juelz Santana was 18 years old, then came February 18th, which makes Juelz turn 19 making songs with The Diplomats. The third neff to be signed to Cam's label under Roc A Fella. In 2003, he was 20 years old coming out with his own singles ""Santana's Town"" and ""Down"". So yes, he is born in 1983. He really is, how could he be older then Lloyd Banks? And how could he be 22 when his birthday passed? The homie neff is 23 years old. 1983 - 2006 (Juelz death, god forbid if your thinking about that) equals 23. Go to your caculator and stop changing his year of birth. My god."


'juelz santanas age in juelz santana was years old then came february th which makes juelz turn making songs with the diplomats the third neff to be signed to cam s label under roc a fella in he was years old coming out with his own singles santana s town and down so yes he is born in he really is how could he be older then lloyd banks and how could he be when his birthday passed the homie neff is years old juelz death god forbid if your thinking about that equals go to your caculator and stop changing his year of birth my god'

In [19]:
test_text = corpus[22]
print(test_text)
clear_text(test_text) # Проверяем работоспособность

"

 Snowflakes are NOT always symmetrical! 

Under Geometry it is stated that ""A snowflake always has six symmetric arms."" This assertion is simply not true! According to Kenneth Libbrecht, ""The rather unattractive irregular crystals are by far the most common variety."" http://www.its.caltech.edu/~atomic/snowcrystals/myths/myths.htm#perfection Someone really need to take a look at his site and get FACTS off of it because I still see a decent number of falsities on this page. (forgive me Im new at this and dont want to edit anything)"


'snowflakes are not always symmetrical under geometry it is stated that a snowflake always has six symmetric arms this assertion is simply not true according to kenneth libbrecht the rather unattractive irregular crystals are by far the most common variety someone really need to take a look at his site and get facts off of it because i still see a decent number of falsities on this page forgive me im new at this and dont want to edit anything'

Работает.    
Я определю единую функцию.

In [20]:
lemmatizer = WordNetLemmatizer()

def clear_and_lemmatize_text(text):  
    
    regex =r'[^a-zA-Z\']|(\butc\b|(http://\S+))' # r'[^a-zA-Zа-яА-ЯёЁ]'
    cleaned_text = re.sub(regex, ' ', text.lower())    
    # Tokenize: Split the sentence into words
    word_list = nltk.word_tokenize(cleaned_text)
    # Lemmatize list of words and join
    lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
    return lemmatized_output


Добавим функцию тега, описывающую часть речи. 

In [21]:
# Lemmatize with POS Tag
from nltk.corpus import wordnet
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

In [22]:
print(lemmatizer.lemmatize("are", 'v')) 

be


In [23]:
print(nltk.pos_tag(['feet']))

[('feet', 'NNS')]


In [24]:
def clear_and_lemmatize_text_with_pos_tag(text):    
    regex =r'[^a-zA-Z]|(\butc\b|(http://\S+))' # r'[^a-zA-Zа-яА-ЯёЁ]'
    cleaned_text = re.sub(regex, ' ', text.lower())    
    # Tokenize: Split the sentence into words
    word_list = nltk.word_tokenize(cleaned_text)
    # Lemmatize list of words and join
    lemmatized_output = ' '.join([lemmatizer.lemmatize(w,get_wordnet_pos(w)) for w in word_list])
    return lemmatized_output

In [25]:
%%time
for row in corpus[0:1000]:
#     print(row)
    clear_and_lemmatize_text_with_pos_tag(row)

Wall time: 27.4 s


Если оценивать примерно общее время, то видим:

In [26]:
160*40/60 # минуты

106.66666666666667

100+ минут ожидаемое время. Терпимо.

Операция в ячейке ниже проделана один раз. 

In [27]:
%%time
# corpus_cleaned = []

# for row in corpus:
#     corpus_cleaned.append(clear_and_lemmatize_text_with_pos_tag(row))   
# corpus_cleaned
# d = {'text': corpus_cleaned}
# lemmas = pd.DataFrame(data = d).to_csv('lemmas.csv')

Wall time: 0 ns


In [28]:
print("Исходный текст: \n", corpus[0])
print("Очищенный и лемматизированный текст: \n", clear_and_lemmatize_text_with_pos_tag(corpus[0]))

Исходный текст: 
 Explanation
Why the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27
Очищенный и лемматизированный текст: 
 explanation why the edits make under my username hardcore metallica fan be revert they weren t vandalism just closure on some gas after i vote at new york doll fac and please don t remove the template from the talk page since i m retire now


Попробуем обработать тот же текст без тегов алгоритмом из тренажера

In [29]:
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer(strip_handles=True, reduce_len=True)
def lemm_clear_text(text):
    word_list = tknzr.tokenize(text) # разбиваем входной текст на токены(слова)
    lemm_text = ' '.join([lemmatizer.lemmatize(w) for w in word_list]) # лемматизируем каждое слово и объединяем в строку
    clear_list = re.sub(r'[^A-Za-z\']', ' ', lemm_text) #оставляем только латинские символы
    return " ".join(clear_list.split()) #функция возвращает очищенный и лемматизированный текст в виде строки

In [30]:
%%time
# применим функцию к нашим комментариям и создадим столбец, хранящий лемматизированные предложения
corpus_cleaned_v2 = list(toxic_comments['text'].str.lower().apply(lambda x: lemm_clear_text(x)))

Wall time: 1min 15s


In [31]:
print("Исходный текст: \n", corpus[0])
print("Очищенный и лемматизированный текст: \n", corpus_cleaned_v2[0])

Исходный текст: 
 Explanation
Why the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27
Очищенный и лемматизированный текст: 
 explanation why the edits made under my username hardcore metallica fan were reverted they weren't vandalism just closure on some gas after i voted at new york doll fac and please don't remove the template from the talk page since i'm retired now


In [32]:
d = {'text': corpus_cleaned_v2}
pd.DataFrame(data = d).to_csv('lemmas_v2.csv')

## 1.3 [Создание матрицы TF-IDF](#Content) <a id='1.3'></a>

Загружаем ранее записанный очищенный корпус

In [33]:
clear_lemmas = pd.read_csv('lemmas.csv')

In [34]:
clear_lemmas.head()

,Unnamed: 0,text
0,0,explanation why the edits make under my userna...
1,1,d aww he match this background colour i m seem...
2,2,hey man i m really not try to edit war it s ju...
3,3,more i can t make any real suggestion on impro...
4,4,you sir be my hero any chance you remember wha...


Преобразуем в список в кодировке Unicode.

In [35]:
lemmatized_corpus = list(clear_lemmas['text'].values.astype('U'))

`lemmatized_corpus` - очищенный корпус текстов далее предстоит преобразовать в матричную форму, с которой может работать машина. 

Проэкспериментируем с размерностями биграмм.

Посчитаем число уникальных биграмм (пункт выполнен ради собственного интереса)

In [36]:
%%time
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(ngram_range=(2, 2))
# создайте n-грамму n_gramm, для которой n=2
n_gramm = count_vect.fit_transform(lemmatized_corpus)
print("Размер:", n_gramm.shape)

Размер: (159571, 1870582)
Wall time: 19.3 s


Чудеса размерности матрицы... >1.800.000 уникальных биграмм. Уберем стоп-слова.

In [37]:
from nltk.corpus import stopwords
import nltk

In [38]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pnedviga\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [39]:
stop_words = set(stopwords.words('english'))

In [40]:
print(stop_words)

{'it', 'myself', 'yourselves', 'from', "couldn't", 'will', 'your', "you're", 'very', 'you', 'same', 'these', 'off', 'during', 'herself', "you'll", 'with', 'them', 'been', "you've", 'because', 'i', "that'll", 'how', "aren't", 'll', 'which', 'couldn', 'an', 'before', "haven't", 'can', 'theirs', "mightn't", 'after', 'doesn', 'needn', 'then', 'most', 'out', 'aren', 'both', 'my', "it's", 's', 'was', 'we', 'shan', 'do', 'over', 'now', 'above', 'does', 'being', 'me', 'her', "should've", 'wasn', 'themselves', 'this', 'of', 'again', 'm', 'mightn', 'no', 'to', "didn't", 'just', 'they', 'as', 'yourself', 'shouldn', "isn't", 'had', "won't", 'here', 'a', 'has', 'have', 'isn', 'by', "don't", 'ma', 'below', 'didn', 'are', 'ourselves', 'having', 'wouldn', 'in', 'up', 'about', 'down', 'his', 'be', "weren't", 'through', 'not', "she's", 'once', 'yours', 'while', 'doing', 'only', "needn't", 'each', 'he', 'more', 'such', 'that', "hasn't", "wouldn't", 'haven', 'until', 'd', 'himself', 'were', "doesn't", "sh

In [41]:
%%time
count_vect = CountVectorizer(stop_words=stop_words,ngram_range=(2, 2))
n_gramm = count_vect.fit_transform(lemmatized_corpus)
print("Размер:", n_gramm.shape)

Размер: (159571, 2300282)
Wall time: 17.1 s


Размерность матрицы повысилась (было `(159571, 1870582)`, стало `(159571, 2300282)`). Интересно. Причина должна быть в том, что число сочетаний с стоп-словами меньше ввиду частоты употребления этих слов в биграмме. Убрав их (стоп-слова) мы увеличили число "уникальных" сочетаний

<span style="color:green"> Круто, что разобрался с этим)</span>

Теперь, после небольшой исследовательской части, делим датасет на тестовую и валидационную выборку для формирования матрицы признаков TF-IDF. 

In [67]:
from sklearn.model_selection import train_test_split

In [71]:
(corp_v1_train, corp_v1_test, corp_v2_train,corp_v2_test, toxic_train, toxic_test)= (train_test_split(lemmatized_corpus, 
                                                                                                        corpus_cleaned_v2,
                                                                                                        toxic_comments.toxic,
                                                                                                        test_size=0.2, 
                                                                                                        random_state=124211))

Приступим к вычислению TF-IDF. 

In [72]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [99]:
TfidfVectorizer_v1 = TfidfVectorizer(stop_words=stop_words,min_df = 2)
TfidfVectorizer_v2 = TfidfVectorizer(stop_words=stop_words,min_df = 2)

Почему 2 шт. как пороговое значение? Ответить честно не могу. Идея такая - у нас выборка делится на 2 части: обучающую и тестовую. Соответственно шансов попасть в тестовую выборку при частоте <2 у признака нет.  В таком случае несет ли он смысловую нагрузку?

In [100]:
tf_idf_v1_train = TfidfVectorizer_v1.fit_transform(corp_v1_train)
tf_idf_v1_test = TfidfVectorizer_v1.transform(corp_v1_test)
print("Размер матрицы train:", tf_idf_v1_train.shape)
print("Размер матрицы test:", tf_idf_v1_test.shape)

Размер матрицы train: (127656, 51132)
Размер матрицы test: (31915, 51132)


Неплохо. Уникальных слов всего чуть больше 51 тысячи... Жить стало попроще. На основе данных признаков можно будет приступать к обучению в [главе 2.1](#2.1)

То же самое проделываем с другой версией корпуса текстов.

In [101]:
tf_idf_v2_train = TfidfVectorizer_v2.fit_transform(corp_v2_train)
tf_idf_v2_test = TfidfVectorizer_v2.transform(corp_v2_test)
print("Размер матрицы train:", tf_idf_v2_train.shape)
print("Размер матрицы test:", tf_idf_v2_test.shape) 

Размер матрицы train: (127656, 57478)
Размер матрицы test: (31915, 57478)


Число признаков уже побольше: 57,5 тыс.

## 1.4 [Создание Embeddings на базе BERT](#Content) <a id='1.4'></a>

Насколько информирует нас тренажер, модель BERT позволяет нам закодировать слова с учетом их контекста и смысловой нагрузки. За такую точность нам приходится платить скоростью работы алгоритма. 

Поскольку возможности компьютера не безграничны, нам надо облегчить задачу для машины насколько это возможно:
1. Брать облегченные модели BERT
2. Ограничить размер батча в рамках одной итерации
3. Использовать на "полную" мощность имеющегося "железа". Полагаю видеокарта nVidia будет отличным подспорьем (https://www.kaggle.com/atulanandjha/distilbert-on-gpu-tutorial-classification-problem)
4. Облачные платформы (для себя ставлю в планы). Говорят kaggle дает несколько часов облачных вычислений "в подарок".

Начнем с использования ядер GPU. Мы не будем ждать долго
Приступаем    
https://pytorch.org/?utm_source=Google&utm_medium=PaidSearch&utm_campaign=%2A%2ALP+-+TM+-+General+-+HV+-+RU&utm_adgroup=PyTorch+Installation&utm_keyword=pytorch%20installation&utm_offering=AI&utm_Product=PyTorch&gclid=EAIaIQobChMIrbrkhqSf6gIVDhsYCh3AcA-zEAAYASAAEgJM2fD_BwE

In [158]:
pip install torch===1.5.1 torchvision===0.6.1 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: torchvision===0.6.1 in c:\users\pnedviga\anaconda3\lib\site-packages (0.6.1)



ERROR: Error checking for conflicts.
Traceback (most recent call last):
  File "C:\Users\pnedviga\Anaconda3\lib\site-packages\pip\_vendor\pkg_resources\__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "C:\Users\pnedviga\Anaconda3\lib\site-packages\pip\_vendor\pkg_resources\__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\pnedviga\Anaconda3\lib\site-packages\pip\_vendor\pkg_resources\__init__.py", line 3012, in _parsed_pkg_info
    return self._pkg_info
  File "C:\Users\pnedviga\Anaconda3\lib\site-packages\pip\_vendor\pkg_resources\__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _pkg_info

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\pnedviga\Anaconda3\lib\site-pack

Далее идем согласно статье: https://habr.com/ru/post/498144/

Импортируем библиотеки.

In [159]:
import torch
import transformers
import numpy as np
import transformers as ppb # pytorch transformers

In [160]:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')
# Загрузка предобученной модели/токенизатора 
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Проверим, что там делает с предложениями токенизатор....

In [162]:
toxic_comments['text'][0:2].apply(
  lambda x: tokenizer.encode(x, add_special_tokens=True))

0    [101, 7526, 2339, 1996, 10086, 2015, 2081, 210...
1    [101, 1040, 1005, 22091, 2860, 999, 2002, 3503...
Name: text, dtype: object

In [163]:
len(toxic_comments['text'][0:2].apply(
  lambda x: tokenizer.encode(x, add_special_tokens=True))[0])

68

In [164]:
len(toxic_comments['text'][0:2].apply(
  lambda x: tokenizer.encode(x, add_special_tokens=True))[1])

35

Каждый текст токенизируется в отдельный вектор. Каждое слово получает свой номер токена. Это первый этап.

Токенизируем тексты. Но мы возьмем не всю выборку, а сделаем даунсемплинг для баланса классов

In [165]:
toxic_comments.columns

Index(['text', 'toxic'], dtype='object')

In [166]:
toxic_comment_reduced = toxic_comments.sample(n=30000,random_state=1)
toxic_comment_reduced.toxic.value_counts()

0    26897
1     3103
Name: toxic, dtype: int64

Чтобы наши токены не получились длиннее 512 позиций ставим ограничитель. Иначе выдается предупреждение об общем количестве векторов. 

In [175]:
%%time
tokenized = toxic_comment_reduced['text'].apply(
  lambda x: tokenizer.encode(x, add_special_tokens=True,max_length = 512))

Wall time: 43.4 s


In [176]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)
max_len

512

Далее приводим все вектора к одному размеру. У нас он как раз 512. 

In [177]:
%%time
padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

Wall time: 1.16 s


In [178]:
padded.shape

(30000, 512)

Итак получили прямоугольную матрицу с токенами на каждый пост. Часть из постов обрезана. Таковы ограничения модели и моего железа. 

In [179]:
# создадим маску для важных токенов
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(30000, 512)

In [180]:
attention_mask

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])

In [181]:
from tqdm import notebook

In [182]:
batch_size = 20
embeddings = []
for i in notebook.tqdm(range(100 // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].numpy())

У меня сокрость 2 предложения в секунду. На всю выборку уйдет:

In [183]:
print(f'Ожидаемое время в часах работы машины {(160000/2/60/60)}' )

Ожидаемое время в часах работы машины 22.22222222222222


Не устраивает! Попробуем поработать с графическим ядром.

In [184]:
import time

In [185]:
!nvidia-smi

Sat Jun 27 18:46:53 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 441.45       Driver Version: 441.45       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 105... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   69C    P8    N/A /  N/A |     75MiB /  4096MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [186]:
#initiating Garbage Collector for GPU environment setup
import gc
for obj in gc.get_objects():
    try:
        if torch.is_tensor(obj) or (hasattr(obj, 'data') and torch.is_tensor(obj.data)):
            print(type(obj), obj.size())
    except:
        pass

<class 'torch.Tensor'> torch.Size([20, 768])
<class 'torch.Tensor'> torch.Size([20, 768])
<class 'torch.Tensor'> torch.Size([20, 768])
<class 'torch.Tensor'> torch.Size([20, 512])
<class 'torch.Tensor'> torch.Size([20, 512])
<class 'torch.Tensor'> torch.Size([20, 512, 768])
<class 'torch.Tensor'> torch.Size([20, 768])
<class 'torch.Tensor'> torch.Size([20, 768])
<class 'torch.nn.parameter.Parameter'> torch.Size([30522, 768])
<class 'torch.nn.parameter.Parameter'> torch.Size([512, 768])
<class 'torch.nn.parameter.Parameter'> torch.Size([768])
<class 'torch.nn.parameter.Parameter'> torch.Size([768])
<class 'torch.nn.parameter.Parameter'> torch.Size([768])
<class 'torch.nn.parameter.Parameter'> torch.Size([768])
<class 'torch.nn.parameter.Parameter'> torch.Size([768])
<class 'torch.nn.parameter.Parameter'> torch.Size([768])
<class 'torch.nn.parameter.Parameter'> torch.Size([768])
<class 'torch.nn.parameter.Parameter'> torch.Size([768])
<class 'torch.nn.parameter.Parameter'> torch.Size([76

In [187]:
torch.cuda.is_available()

True

In [188]:
USE_GPU=True
if USE_GPU and torch.cuda.is_available():
    print('using device: cuda')
    
else:
    print('using device: cpu')

using device: cuda


In [189]:
use_cuda = USE_GPU and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
device

device(type='cuda')

In [190]:
%%time
embeddings = []
if USE_GPU and torch.cuda.is_available():
    print('Using GPU')
    batch_size = 10    
    model.to(device)
    for i in notebook.tqdm(range(100 // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]).to(device) #LongTensor
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)]).to(device) #LongTensor
        
        with torch.no_grad():
            
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy())

Using GPU



Wall time: 8.53 s


In [195]:
print(f'Ожидаемое время в часах работы машины {(160000/100*5/60/60)}' )

Ожидаемое время в часах работы машины 2.2222222222222223


<a id='features_BERT'></a>
Это победа, друзья! Значительное увеличение производительности! Почувствуйте разницу! Большую скорость даст только производительная видеокарта или облачные вычисления. 

Мы работаем сейчас на части общей выборки. Думаю число текстов в 30000 вполне должно хватить, чтобы оценить точность потенциальной модели

In [192]:
%%time
embeddings = []
if USE_GPU and torch.cuda.is_available():
    print('Using GPU')
    batch_size = 20    
    model.to(device)
    for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]).to(device) #LongTensor
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)]).to(device) #LongTensor
        
        with torch.no_grad():
            
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy())

Using GPU



Wall time: 1h 21min 24s


In [193]:
features_BERT = np.concatenate(embeddings)

Мы подготовили признаки для обучения модели: эмбеддинги в количестве 10 тыс. Остается проверить, насколько хорошо они отражают реальное содержимое текста. Об этом мы узнаем в продолжении в [главе 2.3](#2.3)

<span style="color:green"> Очень круто, что разобрался как использовать Берт. Это очень классный уровень</span>

# 2. [Обучение](#Content) <a id='2'></a>

## 2.1 [Логистическая регрессия на признаках TF-IDF](#Content) <a id='2.1'></a>

In [97]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

In [106]:
%%time
log_regression = LogisticRegression(class_weight = 'balanced',max_iter=200)
log_regression.fit(tf_idf_v1_train, toxic_train)
f1_score(toxic_test,log_regression.predict(tf_idf_v1_test))

Wall time: 3.23 s


0.7426067907995619

In [118]:
%%time
log_regression_v2 = LogisticRegression(class_weight = 'balanced',max_iter=400)
log_regression_v2.fit(tf_idf_v2_train, toxic_train)
f1_score(toxic_test,log_regression_v2.predict(tf_idf_v2_test))

Wall time: 3.5 s


0.7472406181015452

Заданная точность метрики **НЕ достигнута**. Но мы обучим и другие модели.

## 2.2 [LightGBM на признаках TF-IDF](#Content) <a id='2.2'></a>

In [ ]:
pip install lightgbm

In [108]:
import lightgbm as lgb

In [109]:
from lightgbm.sklearn import LGBMClassifier

In [110]:
LGBMCl = LGBMClassifier(random_state = 12345, max_depth = 10, n_estimators = 50, metric = 'f1_score')

In [111]:
%%time
LGBMCl.fit(tf_idf_v2_train, toxic_train)

Wall time: 6.82 s


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=10,
               metric='f1_score', min_child_samples=20, min_child_weight=0.001,
               min_split_gain=0.0, n_estimators=50, n_jobs=-1, num_leaves=31,
               objective=None, random_state=12345, reg_alpha=0.0,
               reg_lambda=0.0, silent=True, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)

In [112]:
f1_score(toxic_test,LGBMCl.predict(tf_idf_v2_test))

0.6392099180500105

Пока что печально проигрывает линейной регрессии.

Далее немного оптимизируем подход к подбору гиперпараметров:
https://towardsdatascience.com/an-example-of-hyperparameter-optimization-on-xgboost-lightgbm-and-catboost-using-hyperopt-12bc41a271e
https://medium.com/district-data-labs/parameter-tuning-with-hyperopt-faa86acdfdce

In [ ]:
pip install hyperopt

In [113]:
from hyperopt import hp, fmin, tpe, STATUS_OK, STATUS_FAIL, Trials, space_eval
import numpy as np

In [114]:
# LightGBM parameters
lgb_clf_params = {
    'learning_rate':    hp.choice('learning_rate',    np.arange(0.05, 0.31, 0.05)),
    'max_depth':        hp.choice('max_depth',        np.arange(5, 33, 3, dtype=int)),
    'min_child_weight': hp.choice('min_child_weight', np.arange(1, 8, 1, dtype=int)),
    'colsample_bytree': hp.choice('colsample_bytree', np.arange(0.3, 0.8, 0.1)),
    'subsample':        hp.uniform('subsample', 0.8, 1),
    'n_estimators':     100,
}
lgb_fit_params = {
    'eval_metric': 'binary',
    'early_stopping_rounds': 10,
    'verbose': False
}
lgb_para = dict()
lgb_para['clf_params'] = lgb_clf_params
lgb_para['fit_params'] = lgb_fit_params
lgb_para['loss_func' ] = lambda y, pred: (-f1_score(y, pred)) # np.sqrt(mean_squared_error(y, pred))

In [115]:
class HPOpt(object):

    def __init__(self, x_train, x_test, y_train, y_test):
        self.x_train = x_train
        self.x_test  = x_test
        self.y_train = y_train
        self.y_test  = y_test

    def process(self, fn_name, space, trials, algo, max_evals):
        fn = getattr(self, fn_name)
        try:
            result = fmin(fn=fn, space=space, algo=algo, max_evals=max_evals, trials=trials)
        except Exception as e:
            return {'status': STATUS_FAIL,
                    'exception': str(e)}
        return result, trials
    
    def lgb_reg(self, para):
        reg = lgb.LGBMClassifier(**para['clf_params'])
        return self.train_reg(reg, para)

    def train_reg(self, reg, para):
        reg.fit(self.x_train, self.y_train,
                eval_set=[(self.x_train, self.y_train), (self.x_test, self.y_test)],
                **para['fit_params'])
        pred = reg.predict(self.x_test)
        loss = para['loss_func'](self.y_test, pred)
        return {'loss': loss, 'status': STATUS_OK}

Разобьем на тренировочную/валидационную выборки

In [120]:
(tf_idf_v1_tr, tf_idf_v1_val, tf_idf_v2_tr,tf_idf_v2_val, toxic_tr, toxic_val) = (train_test_split(tf_idf_v1_train,
                                                                                             tf_idf_v2_train,
                                                                                             toxic_train,
                                                                                             test_size=0.2, 
                                                                                             random_state=124211))

Далее загружаем данные и начинаем оптимизацию

In [132]:
%%time
obj_v1 = HPOpt(tf_idf_v1_tr, 
            tf_idf_v1_val,
            toxic_tr,
            toxic_val)

Wall time: 0 ns


In [133]:
%%time
lgb_opt_v1 = obj_v1.process(fn_name='lgb_reg', 
                      space=lgb_para, 
                      trials=Trials(), 
                      algo=tpe.suggest, 
                      max_evals=10)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [01:36<00:00,  9.60s/trial, best loss: -0.768667957822251]
Wall time: 1min 36s


Получены следующие оптимальные параметры по TF-IDF версии 1:

In [153]:
optimum_params_v1 = space_eval(lgb_clf_params, lgb_opt_v1[0])
optimum_params_v1

{'colsample_bytree': 0.7000000000000002,
 'learning_rate': 0.3,
 'max_depth': 29,
 'min_child_weight': 2,
 'n_estimators': 100,
 'subsample': 0.8329239159623381}

In [154]:
LGBMCl_opt_v1 = LGBMClassifier(**optimum_params_v1)#random_state = 12345, 

In [156]:
%%time
LGBMCl_opt_v1.fit(tf_idf_v1_train, toxic_train)

Wall time: 15.1 s


LGBMClassifier(boosting_type='gbdt', class_weight=None,
               colsample_bytree=0.7000000000000002, importance_type='split',
               learning_rate=0.3, max_depth=29, min_child_samples=20,
               min_child_weight=2, min_split_gain=0.0, n_estimators=100,
               n_jobs=-1, num_leaves=31, objective=None, random_state=None,
               reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=0.8329239159623381, subsample_for_bin=200000,
               subsample_freq=0)

In [157]:
f1_score(toxic_test,LGBMCl_opt_v1.predict(tf_idf_v1_test))

0.7682119205298014

Уровень метрики f1 на тестовой выборке достигнут! `0.768`

Пробуем тот же метод на TF-IDF_V2

In [138]:
%%time
obj_v2 = HPOpt(tf_idf_v2_tr, 
            tf_idf_v2_val,
            toxic_tr,
            toxic_val)

Wall time: 0 ns


In [148]:
%%time
lgb_opt_v2 = obj_v2.process(fn_name='lgb_reg', 
                      space=lgb_para, 
                      trials=Trials(), 
                      algo=tpe.suggest, 
                      max_evals=10)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [01:35<00:00,  9.53s/trial, best loss: -0.7706382051840557]
Wall time: 1min 35s


Получены следующие оптимальные параметры для версии TF-IDF_v2

In [149]:
optimum_params_v2 = space_eval(lgb_clf_params, lgb_opt_v2[0])
optimum_params_v2

{'colsample_bytree': 0.4,
 'learning_rate': 0.25,
 'max_depth': 32,
 'min_child_weight': 1,
 'n_estimators': 100,
 'subsample': 0.9361514243982431}

In [150]:
LGBMCl_opt_v2 = LGBMClassifier(**optimum_params_v2)#random_state = 12345, 

In [151]:
%%time
LGBMCl_opt_v2.fit(tf_idf_v2_train, toxic_train)

Wall time: 15.5 s


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.4,
               importance_type='split', learning_rate=0.25, max_depth=32,
               min_child_samples=20, min_child_weight=1, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=0.9361514243982431, subsample_for_bin=200000,
               subsample_freq=0)

In [152]:
f1_score(toxic_test,LGBMCl_opt.predict(tf_idf_v2_test))

0.7710974284679464

Новая модель дает f1 меру `0.771` Уже чуть больше. 

## 2.3 [Логистическая регрессия на эмбеддингах из BERT](#Content) <a id='2.3'></a>

Признаки подготовлены [здесь](#features_BERT)    
Нам предстоит разделить нашу выборку на тестовый и тренировочный датасет. 

In [196]:
features_BERT_train, features_BERT_test, target_BERT_train, target_BERT_test = (train_test_split(features_BERT,
                                                                                                 toxic_comment_reduced.toxic,
                                                                                                 test_size=0.2,
                                                                                                 random_state=1))

In [197]:
target_BERT_test.shape

(6000,)

In [198]:
%%time
log_regression_BERT = LogisticRegression(class_weight = 'balanced',max_iter=500)
log_regression_BERT.fit(features_BERT_train, target_BERT_train)

Wall time: 10.3 s


C:\Users\pnedviga\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=500, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [199]:
f1_score(target_BERT_test,log_regression_BERT.predict(features_BERT_test))

0.6700443318556049

In [200]:
f1_score(target_BERT_train,log_regression_BERT.predict(features_BERT_train))

0.7184555984555986

Немного обескуражен результатом. Потрачено гораздо больше времени и при этом получен худший результат на тектовой выборке. 
Пока остановлюсь на этом в своих изысканиях и подведу итоги в следующей главе.

## 2.4 [LightGBM на эмбеддингах из BERT](#Content) <a id='2.4'></a>

Параметры оставим без изменения для Hyperopt

In [201]:
%%time
obj_BERT = HPOpt(features_BERT_train, 
            features_BERT_test,
            target_BERT_train,
            target_BERT_test)

Wall time: 0 ns


In [203]:
%%time
lgb_opt_BERT = obj_BERT.process(fn_name='lgb_reg', 
                      space=lgb_para, 
                      trials=Trials(), 
                      algo=tpe.suggest, 
                      max_evals=15)


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [01:19<00:00,  5.33s/trial, best loss: -0.6881516587677725]
Wall time: 1min 19s


Получены следующие оптимальные параметры

In [204]:
optimum_params_BERT = space_eval(lgb_clf_params, lgb_opt_BERT[0])
optimum_params_BERT

{'colsample_bytree': 0.5,
 'learning_rate': 0.1,
 'max_depth': 23,
 'min_child_weight': 6,
 'n_estimators': 100,
 'subsample': 0.856380335487004}

In [205]:
LGBMCl_BERT_opt = LGBMClassifier(**optimum_params_BERT)

In [206]:
%%time
LGBMCl_BERT_opt.fit(features_BERT_train, target_BERT_train)

Wall time: 5.62 s


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.5,
               importance_type='split', learning_rate=0.1, max_depth=23,
               min_child_samples=20, min_child_weight=6, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=0.856380335487004, subsample_for_bin=200000,
               subsample_freq=0)

In [207]:
f1_score(target_BERT_test,LGBMCl_BERT_opt.predict(features_BERT_test))

0.6850618458610848

Конечно это не тот результат, за который боролись! `0.69` - совсем немного.

Буду рад любым рекомендациям. Результат явно может быть выше

# 3. [Выводы](#Content) <a id='3'></a>

### Краткий обзор проведенной работы
Что выполнили, сделали, рассчитали;
1. Был открыт и изучен датасет. 
2. Мы рассмотрели различного рода преобразования текста в численные признаки:
    - Создание n-грамм
    - TF-IDF - частота употребления слова в совокупности с их важностью. 
    - Создание признаков (эмбеддингов) на базе BERT. 
3. Обучено два вида моделей на базе двух наборов признаков:
    - LightGBM
    - LogisticRegression
4. Получены результаты классификации. Не все из них удовлетворяют заданным параметрам точности.    
----
### Главные выводы 
---
Нашей задачей было подготовить и найти модель с устраивающим нас показателем точности.
Подбеду тут одержала модель LightGBM на признаках TF-IDF с показателем `f1 = 0.771`. Следующей по качеству оказалась линейная регрессия с `f1 = 0.747`   

Embeddings от BERT к сожалению и близко не дали таких результатов. По идее модель от BERT должна быть точнее, поскольку более корректно сопоставляет свойства (смысл слова) с численным представлением. 

Возможные причины провала эмбеддингов:
- Малая выборка (сделан downsapling). Мало примеров. Есть большая вероятность, что понятие, встретившееся в тестовой выборке не встречалось в обучающей. 
- Несбалансированные классы в обучающей выборке.
- Возможно повлияла размерность матрицы эмбеддингов - мы выполнили "усечение", что равносильно потере части информации. Соответственно осталось меньше "пересекающихся эмбеддингов" в тренировочной vs. тестовой выборке.    
---
### Рекомендации
- Вероятно стоит получить embeddings для всего набора данных и уже после обучать модель. 
- Одновременно не следует "резать" признаки. У нас ограничения от модели составили 512 штук. Логичным шагом будет разбить каждое высказывание на предложения. 
- Следует выбирать подходящую по мощности машину или облачное решение для работы с текстами.
---
### Итог
Для себя вынес следующее:
в ML тексты пожалуй самое сложное, с чем можно работать. Думаю сложнее только декомпозиция записи речи. Там можно встретить ряды в совокупности с текстами. 
Вместе с тем тема очень интересная. Думаю поискать литературу по тематике.  
***UPD 04.11.2020: купил книжечку 2020 года Орельена Жерона "Прикладное машинное обучение..." (2е издание)***